In [ ]:
import os
import tell
import pandas as pd

In [ ]:
startyear={"historical":2000,'future':2020}
endyear={"historical":2019,'future':2059}

In [ ]:
mdir='/orcd/nese/mhowland/001/lyqiu/GODEEP/'

# MLP training

In [ ]:
# Run the MLP training step for a single BA (i.e., "region"):
#By default the MLP models are trained on data from 2016-2018 and evaluated using data from 2019. 
prediction_df, validation_df = tell.train(region = 'CISO',
                                          data_dir = f'{mdir}/Demand/Demand_TELL/inputs/compiled_historical_data/')

# View the head of the prediction dataframe that contains the time-series of projected load in the evaluation year:
display(prediction_df.head(10))

# View validation dataframe that contains error statistics for the trained model:
validation_df

# MLP model projection


In [ ]:
# Run the MLP prediction step for the list of BAs using parallel processing streams:
ba_abbrev_list = tell.get_balancing_authority_to_model_dict().keys()
clisce='rcp85hotter'
popsce='_ssp5'

if clisce=='historical':
    sy=startyear['historical']
    ey=endyear['historical']
else:
    sy=startyear['future']
    ey=endyear['future']
for iy in range(sy,ey+1):
    pdf = tell.predict_batch(target_region_list = ba_abbrev_list,
                            year = iy,
                            data_dir = f"{mdir}/TGW/Meteorology/{clisce}{popsce}/",
                            datetime_field_name = 'Time_UTC',
                            save_prediction = True,
                            prediction_output_directory = "{mdir}/Demand/Demand_TELL/outputs/mlp_output/%s%s"(clisce,popsce),
                            n_jobs = -1)

# detrend data

In [ ]:
for sce in ['historic','rcp85hotter_ssp5','rcp45hotter_ssp5']:
    if sce=='historic':
        sy=1981
        ey=2020
    else:
        sy=2040
        ey=2060
    for ISO in ['ISNE','CISO','ERCO']:
        df=pd.DataFrame()
        for iy in range(sy,ey):
            a=pd.read_csv(f'{mdir}/TGW/Meteorology/ISOmean/{sce}/{ISO}_WRF_Hourly_Mean_Meteorology_{iy}.csv')
            df=pd.concat([df,a])
        df=df.set_index('Time_UTC')
        mean=df.mean().to_frame().T
        mean.to_csv(f'{mdir}/TGW/Meteorology/ISOmean/{sce}/{ISO}_WRF_Hourly_Mean_Meteorology.csv')


In [ ]:
for sce in ['rcp85hotter_ssp5','rcp45hotter_ssp5']:
    dir=f"{mdir}/TGW/Meteorology/ISOmean_detrend/{sce}/"
    if not os.path.exists(dir):
        os.makedirs(dir)
    for ISO in ['ISNE','CISO','ERCO']:
        hist_mean=pd.read_csv(f'{mdir}/TGW/Meteorology/ISOmean/historic/{ISO}_WRF_Hourly_Mean_Meteorology.csv')
        future_mean=pd.read_csv(f'{mdir}/TGW/Meteorology/ISOmean/{sce}/{ISO}_WRF_Hourly_Mean_Meteorology.csv')    
        correction_add=hist_mean-future_mean
        correction_mul=hist_mean/future_mean
        for iy in range(2040,2060):
            a=pd.read_csv(f'{mdir}/TGW/Meteorology/ISOmean/{sce}/{ISO}_WRF_Hourly_Mean_Meteorology_{iy}.csv')
            a['T2']=a['T2']+correction_add['T2'].values[0]
            a['Q2']=a['Q2']+correction_add['Q2'].values[0]
            a['WSPD']=a['WSPD']+correction_add['WSPD'].values[0]
            a['SWDOWN']=a['SWDOWN']*correction_mul['SWDOWN'].values[0]
            a.to_csv(f'{dir}/{ISO}_WRF_Hourly_Mean_Meteorology_{iy}.csv')


In [ ]:
for sce in ['rcp85hotter_ssp5','rcp45hotter_ssp5']:
    for ISO in ['ISNE','CISO','ERCO']:
        for iy in range(2040,2060):
            pdf = tell.predict(region=ISO,
                                    year = iy,
                                    data_dir = f"{mdir}/TGW/Meteorology/ISOmean_detrend/{sce}/",
                                    datetime_field_name = 'Time_UTC',
                                    save_prediction = True,
                                    prediction_output_directory = f"{mdir}/Demand/Demand_TELL/outputs/mlpdr/{sce}")

# Validation

In [10]:
demanddir=f"{mdir}/Demand/Demand_TELL/outputs/mlp_output/"

In [ ]:
ISOs=['ISONE']
ISOnames={'ISONE':'ISNE','CAISO':'CISO','ERCOT':'ERCO'}
period="historic"
for ISO in ISOs:
    obs=pd.read_csv("/pool001/lyqiu/Siting_Optimization/Qiu_etal_2024_ERCOT/Demand/Demand_%s.csv"%ISO)
    obs['Time']=pd.to_datetime(obs['Time'])
    obs=obs.set_index('Time')
    obs=obs.rename(columns={'Demand':'Obs'})
    # obs=pd.read_csv(f"{mdir}/Demand/Demand_TELL/inputs/historical_ba_load/{ISOnames[ISO]}_hourly_load_data.csv")
    # obs['Time']=pd.to_datetime(obs['Year'].astype(str)+'-'+obs['Month'].astype(str)+'-'+obs['Day'].astype(str)+' '+obs['Hour'].astype(str)+':00:00')    
    # obs=obs.set_index('Time')
    # obs=obs.rename(columns={'Adjusted_Demand_MWh':'Obs'})
    for iy in range(2016,2017):
        telldata=pd.read_csv(demanddir+"%s/%d/%s_%d_mlp_output.csv"%(period,iy,ISOnames[ISO],iy))
        telldata['Time']=pd.to_datetime(telldata['Time_UTC'])
        telldata=telldata.rename(columns={'Load':'MLP'})
        obs_year=obs.loc[str(iy)]
        validation=pd.merge(telldata,obs_year,left_on='Time',right_index=True,how='inner')
        validation['error']=(validation['MLP']-validation['Obs'])/validation['Obs']*100
        meanbias=validation['error'].mean()
        mae=validation['error'].abs().mean()
        mse=(validation['error']**2).mean()
        corr=validation['MLP'].corr(validation['Obs'])
        df=pd.DataFrame({'ISO':[ISO],'Year':[iy],'Mean Bias':[meanbias],'Mean Absolute Error':[mae],'Mean Square Error':[mse],'Correlation':[corr]})
        display(df)

,ISO,Year,Mean Bias,Mean Absolute Error,Mean Square Error,Correlation
0,ISONE,2016,-1.385641,6.898015,66.679286,0.920806
